# Road Word

In [40]:
import tensorflow as tf
import cv2 
import csv
import os
import numpy as np
print(tf.__version__)
from shutil import copyfile
import glob

1.11.0


## Arrange data

In [43]:
if not os.path.isdir('./Data/train'):
    os.makedirs('./Data/train')
if not os.path.isdir('./Data/val'):
    os.makedirs('./Data/val')
if not os.path.isdir('./Data/test'):
    os.makedirs('./Data/test')

directories = os.listdir('./Data/images/')
print('Number of classes: ' + str(len(directories)))
with open('./Data/data_labels.csv', mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for directory in directories:
        if not os.path.isdir('./Data/train/' + directory + '/'):
            os.makedirs('./Data/train/' + directory + '/')
        if not os.path.isdir('./Data/val/' + directory + '/'):
            os.makedirs('./Data/val/' + directory + '/')
        if not os.path.isdir('./Data/test/' + directory + '/'):
            os.makedirs('./Data/test/' + directory + '/')
        for i, file in enumerate(glob.glob('./Data/images/' + directory + '/*.jpg')):
            csv_writer.writerow(['./Data/images/' + directory + '/' + file.split("/")[-1], directory])
            if (i % 4 == 0):
                copyfile(('./Data/images/' + directory + '/' + file.split("/")[-1]), './Data/val/' + directory + '/' + file.split("/")[-1])
            elif (i % 5 == 0):
                copyfile(('./Data/images/' + directory + '/' + file.split("/")[-1]), './Data/test/' + directory + '/' + file.split("/")[-1])
            else:
                copyfile(('./Data/images/' + directory + '/' + file.split("/")[-1]), './Data/train/' + directory + '/' + file.split("/")[-1])

Number of classes: 3


In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 256, 256
train_data_dir = "./Data/train"
validation_data_dir = "./Data/val"
nb_train_samples = 4125
nb_validation_samples = 466 
batch_size = 16
epochs = 10

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

"""
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)       295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_conv4 (Conv2D)        (None, 64, 64, 256)       590080    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 32, 32, 256)       0         
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 32, 32, 512)       1180160   
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_conv4 (Conv2D)        (None, 32, 32, 512)       2359808   
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 16, 16, 512)       0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_conv4 (Conv2D)        (None, 16, 16, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 8, 8, 512)         0         
=================================================================
Total params: 20,024,384.0
Trainable params: 20,024,384.0
Non-trainable params: 0.0
"""

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)


valid_datagen = ImageDataGenerator(
rescale = 1./255,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
)

train_generator = train_datagen.flow_from_directory(
    directory=r"./Data/train/",
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

valid_generator = valid_datagen.flow_from_directory(
    directory=r"./Data/val/",
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=r"./Data/test/",
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=32,
    class_mode=None,
    shuffle=False,
    seed=42
)

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_final.summary()

# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = valid_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

score = model_final.evaluate_generator(generator=valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:84: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 777 images belonging to 3 classes.
Found 326 images belonging to 3 classes.
Found 193 images belonging to 3 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
___________________

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:154: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:154: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=10, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=128, validation_steps=466)`


Epoch 1/10
128/128 [==============================] - 328s 3s/step - loss: 0.7959 - acc: 0.6596 - val_loss: 0.4510 - val_acc: 0.8205

Epoch 00001: val_acc improved from -inf to 0.82048, saving model to vgg16_1.h5
Epoch 2/10
108/128 [========================>.....] - ETA: 19s - loss: 0.4371 - acc: 0.8345